In [1]:
import sys

sys.path.append( '../slevelsutil/' )

import df_util, file_util
import round_track_util as rutil
from Instance import Instance

### 2) Loading the instance data

In [2]:
filter_instance = {"instance_name":"SCENARIO"} #, "customer_segmentation":"X", "service_rate": "S1"}
test_case = "standard_vs_enforce_BB"
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/day/enforce_sl_all_hierachical_scenarios.json"
data_dict_path = "../data/dictionary/request_track_data_dictionary.json"

instance_case = Instance(test_case, instance_settings_path, data_dict_path)
# tripdata = ResultTripServiceLevel(instance)


import os.path
import pandas as pd
from pprint import pprint


# Instance settings
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/rebalancing/instance_settings_test_rebalancing.json"
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/profile_time_less_waiting_relax.json"
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/enforce_service_levels.json"
#instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/rebalancing/no_rebalancing_fast.json"


dict_sl_status = {"MET":"Met", "UNMET":"Unmet"}
dict_sq_class = {"A":"Business", "B":"Standard", "C":"Low-cost"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["Business", "Standard", "Low-cost"], ordered=True)



### 2) Loading the instance data

In [3]:
instances_dic = file_util.load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]

request_log_folder = result_folder + "request_track/"
print(f"Request log folder: {request_log_folder}")

round_log_folder = result_folder + "round_track/"
vehicle_status_graph_folder = result_folder + "vehicle_status/"

# Folder where instances are located
instances_folder = instances_dic["instances_folder"]

# File name aggregated data
instance_name = instances_dic["instance_name"]

print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

Request log folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/day_enforce_sl/request_track/
########### INSTANCE SETTINGS ##################################################
{'adjacency_matrix_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/network_info/adjacency_matrix.csv',
 'distances_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_m.csv',
 'durations_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_duration_s.csv',
 'hiring_config': [{'name': 'method_hire_from_regional_center'}],
 'instance_description': 'Add auxiliary variable (greater equal)',
 'instance_name': 'SCENARIO',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/day_enforce_sl/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_duration',
            'CS': 'customer_segmentation',


### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output = 
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [4]:
from functools import partial

# Example
test_label = "IN-SLBENCH_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S1_VH_SD_RE-OP_OPT-JAVIER"

# Create simplified signature using labels read
get_settings_dict_from_label = partial(Instance.get_instance_settings,  instances_dic["labels"])
get_settings_dict_from_label(test_label)

Loading settings from IN-SLBENCH_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S1_VH_SD_RE-OP_OPT-JAVIER.
VH
SD


{'instance_name': 'SLBENCH',
 'batch_duration': '30',
 'simulation_time': '86400',
 'max_requests': '1000',
 'initial_fleet': '1000',
 'max_capacity': '4',
 'customer_segmentation': 'BB',
 'contract_duration': '0',
 'service_rate': 'S1',
 'allow_vehicle_hiring': True,
 'allow_service_deterioration': True,
 'rebalance': 'OP',
 'matching': 'JAVIER'}

### Aggregate results (folder round_track)

Instance fields in `round_track` folder. Every line is a snapshot of a environment round of 30 seconds (first column is round `timestamp`):

#### Request status per round
* `waiting`
* `finished`
* `denied`
* `n_requests`

#### Freelance vehicles per round
* `hired_vehicles`
* `deactivated_vehicles`

#### Vehicle status per round
* `active_vehicles`
* `enroute_count`
* `parked_vehicles`
* `origin_vehicles`
* `rebalancing`
* `stopped_rebalancing`
* `idle`
* `picking_up`
* `O1,O2,O3,O4`
* `V1,V2,V3,V4`
* `distance_traveled_cruising`
* `distance_traveled_loaded`
* `distance_traveled_rebalancing`
* `run_time`

#### Vehicle status per round (seats)
* `seat_count`
* `picking_up_seats`
* `rebalancing_seats`
* `empty_seats`
* `total_capacity`

#### Service quality
* `pk_delay`
* `total_delay`
* `A_pk,A_dp,A_count,A_unmet_slevels`
* `B_pk,B_dp,B_count,B_unmet_slevels`
* `C_pk,C_dp,C_count,C_unmet_slevels`

In [5]:
def get_results_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}round_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="timestamp",  parse_dates = True)

    # Number of requests
    total_requests = df["n_requests"].sum()
    serviced = df["finished"][-1]
    denied = df["denied"][-1]
    
    # Service quality
    sq_classes = ["A", "B", "C"]
    sq_settings = ["pk", "dp", "count", "unmet_slevels"]
    # e.g., ['A_pk', 'A_dp', 'A_count', 'A_unmet_slevels', 'B_pk', 'B_dp', 'B_count', 'B_unmet_slevels', 'C_pk', 'C_dp', 'C_count', 'C_unmet_slevels']
    sq_user_labels = ["{}_{}".format(sq_class, sq_setting) for sq_class in sq_classes for sq_setting in sq_settings]
    sq_user_dic = {l:df[l][-1] for l in sq_user_labels}
    
    # Distance traveled
    distance_cruising = df["distance_traveled_cruising"][-1]
    distance_loaded = df["distance_traveled_loaded"][-1]
    distance_rebalancing = df["distance_traveled_rebalancing"][-1]
    distance_total = distance_cruising + distance_loaded + distance_rebalancing

    # Pickup
    avg_pk_delay = df["pk_delay"].mean()
    avg_ride_delay = df["total_delay"].mean()

    # Separate occupancy labels (e.g., O1, O2, O3, etc.)
    occupancy_labels = [col for col in list(df) if 'O' in col]
    status_labels = [ "idle", "picking_up"] + occupancy_labels
    
    # Get fleet makeup
    fleet_makeup_labels = [col for col in list(df) if 'V' in col]
    fleet_makeup = {mk:df[mk][-1] for mk in fleet_makeup_labels}
    total_seats = {"seats_" + k:int(k[1:]) * v for k,v in fleet_makeup.items()}
    total_seats["total_seats"] = sum(total_seats.values())
    max_hired = df["denied"].max()

    # Filter occupancy columns
    df_occupancy = df[occupancy_labels]
    
    # Build fleet status
    df_status = pd.DataFrame(df_occupancy)
    df_status["picking_up"] = df["picking_up"]
    df_status["idle"] = df["idle"]
    
    # Smooth values
    df_occupancy = df_occupancy.rolling(window=24).mean()

    # Runtime
    time_ride_matching_ms = df["time_ride_matching_ms"].sum()
    time_update_fleet_status_ms = df["time_update_fleet_status_ms"].sum()
    time_vehicle_rebalancing_ms = df["time_vehicle_rebalancing_ms"].sum()
    total_runtime = time_ride_matching_ms + time_vehicle_rebalancing_ms + time_vehicle_rebalancing_ms
    
    # Dictionary of agreggate data
    dic_agreggate_data = {"serviced": "{:.2%}".format(serviced/total_requests),
                          "denied": "{:.2%}".format(denied/total_requests),
                          'max_hired': max_hired,
                          "total_requests": total_requests,
                          "avg_pk_delay": "{:.2f}".format(avg_pk_delay),
                          "avg_ride_delay": "{:.2f}".format(avg_ride_delay),
                          "total_runtime": "{:.2f}".format(total_runtime/1000/60),
                          "distance_cruising": "{:.2%}".format(distance_cruising/distance_total),
                          "distance_loaded": "{:.2%}".format(distance_loaded/distance_total),
                          "distance_rebalancing": "{:.2%}".format(distance_rebalancing/distance_total),
                          "distance_total": distance_total//1000}
    
    # All data
    dic_ag = {**dic_agreggate_data, **fleet_makeup}
    dic_ag = {**dic_ag, **total_seats}
    dic_ag = {**dic_ag, **sq_user_dic}

    return dic_ag

In [6]:
# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)
print(f"Reading {len(instance_file_names)} files from '{request_log_folder}'.")

# Instance Info (key = instance name)
dic_all_instance_settings = dict()
dic_all_round_track = dict()
dic_all_request_track = dict()

print(instance_file_names)
for file_name in instance_file_names:
    print(f"File name: {file_name}")
    try:
        instance, extension = file_name.split(".")
        print("  - Processing", instance)
    except:
        continue

    instance_settings = get_settings_dict_from_label(instance)

    rutil.plot_vehicle_status_graph(
        round_log_folder,
        vehicle_status_graph_folder,
        instance,
        replace=True,
        smooth=0,
        # label_y = ([750, 1000, 1250, 1500, 1750, 2000], [750, 1000, 1250, 1500, 1750, 2000], "xx-large"),
        #  label_y = ([0, 250, 500, 750, 1000, 1250], [0, 250, 500, 750, 1000, 1250], "xx-large"),
        # label_y = ([0, 25, 50], [0, 25, 50], "xx-large"),
        label_y = ([0, 250, 500, 750, 1000, 1250, 1500, 1750, 2000], [0, 250, 500, 750, 1000, 1250, 1500, 1750, 2000], "xx-large"),
        print_details=False,
        context_font_scale = 1.3,
        # size_inches = (21,3),
        size_inches = (9, 9),
        fontsize="xx-large",
        fontsize_label="xx-large",
        # custom_tw = ('2011-02-01T00:00:00', '2011-02-08T00:00:00'),
        custom_tw = ('2011-02-01T18:00:00', '2011-02-01T21:00:00'),
        x_data_format = ('1h', '%H'),
        color_bg = "#C0C0C0",
        horizontal_legend = True,
        day_sep_config = (dict(linewidth=6, color='white', alpha=0.4), dict(linewidth=1, color='black', linestyle='--')),
        fig_type = 'png',
        print_details_axis = ("Time (h)", "#Vehicles"),
        input_dic = instance_settings
    )

Reading 15 files from 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/day_enforce_sl/request_track/'.
['backup', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV.csv', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-JAVIERSL.csv', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-ERTV.csv', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-JAVIERSL.csv', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV.csv', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S0_VH_SD_RE-OP_OPT-JAVIERSL.csv', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV.csv', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S10_VH_SD_RE-OP_OPT-JAVIERSL.csv', 'IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S8_VH_SD_RE-OP_OPT-ERTV.csv', 'IN-SCENARIO_BA-30_S

<Figure size 648x648 with 0 Axes>

### Comparing hiring for scenario S2 and Segmentation BB

### Read all files in folder (whole week)

In [7]:
# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)


# Instance Info (key = instance name)
dic_all_instance_settings = dict()
dic_all_round_track = dict()
dic_all_request_track = dict()

print("Reading files in folder:", request_log_folder)
cont = 0

general_config = dict(
    replace=True,
    smooth=0,
    print_details=False,
    size_inches = (18,3),
    fontsize="x-large",
    fontsize_label="x-large",
    custom_tw = ('2011-02-01T00:00:00', '2011-02-08T00:00:00'),
    x_data_format = ('6h', '%H'),
    label_y = ([0, 225, 450, 675, 900, 1125, 1350, 1575, 1800, 2025, 2250], [0, 225, 450, 675, 900, 1125, 1350, 1575, 1800,  2025, 2250], "small"),
    color_bg = "#C0C0C0",
    context_font_scale = 1.3,
    horizontal_legend = True,
    day_sep_config = (dict(linewidth=6, color='white', alpha=0.4), dict(linewidth=1, color='black', linestyle='--')),
    fig_type = 'png',
    linewidth = 1,
    input_dic = instance_settings
)

msg_fontsize = 'large'

for file_name in instance_file_names:
    #if cont >1:
    #    break
    cont = cont + 1

    try:
        instance, extension = file_name.split(".")
    except:
        continue

    print("  - Processing", instance)
   
    rutil.plot_vehicle_status_graph(
        round_log_folder,
        vehicle_status_graph_folder,
        instance,
        show_tick_label_x = True,
        show_week_days = True,
        print_details_axis = ("Time (h)", "#Vehicles"),
        **general_config
    )

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/day_enforce_sl/request_track/
  - Processing IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV
Creating figure in 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/day_enforce_sl/vehicle_status/VS_IN-SCENARIO_BA-30_ST-10800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV.png'
TW1(2011-02-01T00:00:00, 2011-02-08T00:00:00)
TW: ('2011-02-01T00:00:00', '2011-02-08T00:00:00')
TW2(2011-02-01T00:00:00, 2011-02-08T00:00:00)
Index(['waiting', 'finished', 'denied', 'n_requests', 'seat_count',
       'picking_up_seats', 'rebalancing_seats', 'empty_seats',
       'total_capacity', 'active_vehicles', 'hired_vehicles',
       'deactivated_vehicles', 'enroute_count', 'pk_delay', 'total_delay',
       'parked_vehicles', 'origin_vehicles', 'rebalancing',
       'stopped_rebalancing', 'idle', 'picking_up', 'O1', 'O2', 'O3', 'O4',
       'V1', 'V2', 'V3', 'V4', 'dista

<Figure size 1296x216 with 0 Axes>

### Create rebalance matrix

In [8]:
  -- -- -- -- MO MO MO MO
CT                       --
CT                       --
CT                       RI
CT                       RI
--                       RI
--                       RI
--                       --
  -- -- UR UR UR UR -- --

SyntaxError: invalid syntax (<ipython-input-8-42f53df68515>, line 1)

In [ ]:
no_rebal = "VS_IN-FIXED_FLEET_NO_REBAL_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB.pdf"

list_rebal = [
    [
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_MO.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_MO_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT.pdf"
    ],
    [
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_MO_RT.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_MO_RT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT.pdf"
    ],
    [
    
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_MO_RT.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_MO_RT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_RT_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_RT.pdf"
    ],
    [
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_MO.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_MO_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE_UR.pdf",
    "IN-rebalancingSettings_BA-30_ST-86400_MR-1000_IF-1000_MC-4_CS-BB_RE.pdf",
    ]
]

for i in range(0, 4):
    for j in range(0, 4):
        instance, extension = list_rebal[i][j].split(".")
        print("  - Processing", instance)

        rutil.plot_vehicle_status_graph(round_log_folder,
                                vehicle_status_graph_folder,
                                instance,
                                replace=True,
                                smooth=1,
                                label_y = ([0, 250, 500, 750, 1000], ["","","","",""], "small"),
                                show_tick_label_x = False,
                                print_details=False,
                                size_inches = (2,2),
                                fontsize="medium",
                                show_week_days = False,
                                show_service_rate = "",
                                fontsize_label="medium",
                                custom_tw = ('2011-02-01T00:00:00', '2011-02-02T00:00:00'),
                                x_data_format = ('4h', '%H'),
                                color_bg = "#C0C0C0",
                                horizontal_legend = True,
                                fig_type = 'pdf',
                                linewidth = 1,
                                print_details_axis = (None, None),
                                input_dic = instance_settings)
        
